# PCAI PDF RAG System

## Complete Implementation of 5 Required Components

This notebook demonstrates a working Retrieval-Augmented Generation system using PDF File

**Components Implemented:**
1. Knowledge Base (PDF)
2. Semantic Layer (sentence-transformers)
3. Retrieval System (FAISS)
4. Augmentation (context enrichment)
5. Generation (answer synthesis)

## Setup: Import All Components

Run this cell first to load all modules.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")

In [2]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [3]:
# Import our 5 components
from knowledge_base import KnowledgeBase
from semantic_layer import SemanticLayer
from retrieval_system import RetrievalSystem
from augmentation import Augmentation
from generation import Generation
print("✅ All components imported successfully!")

✅ All components imported successfully!


## Component 1: Knowledge Base

Load PCAI dataset from pdf.

In [4]:

# Initialize KB with PDF and chunk settings
kb = KnowledgeBase(pdf_path="../data/hpe-pcai.pdf", chunk_size=200, chunk_overlap=50)

# Load PDF pages
documents = kb.load_pdf_data()

# Create sentence-preserving chunks
chunks = kb.create_chunks()

# Inspect stats
stats = kb.get_stats()
stats = kb.get_stats()
print(f"\n📊 Knowledge Base:")
print(f"   Pages: {stats['total_pages']}")
print(f"   Chunks: {stats['total_chunks']}")
print(f"   Source: {stats['source']}")


📚 Knowledge Base initialized

📥 Loading PDF: ../data/hpe-pcai.pdf
✅ Loaded 24 pages from PDF
✅ Created 57 text chunks

📊 Knowledge Base:
   Pages: 24
   Chunks: 57
   Source: ../data/hpe-pcai.pdf


## Component 2: Semantic Layer

Convert text to numerical embeddings for semantic comparison.

In [5]:
import random
import numpy as np

# =========================
# Random Preview: Chunks & Embeddings
# =========================

# Number of random samples to preview
N = 5

# Ensure embeddings exist
try:
    document_embeddings
except NameError:
    # Encode chunks if not done yet
    semantic = SemanticLayer()
    document_embeddings = semantic.encode_documents(kb.chunks)

# Random indices
random_indices = random.sample(range(len(kb.chunks)), min(N, len(kb.chunks)))

print(f"\n📄 Random preview of {N} chunks with embeddings:\n")

for i, idx in enumerate(random_indices):
    chunk = kb.chunks[idx]
    emb = document_embeddings[idx]
    
    print(f"--- Sample {i+1} (Index {idx}) ---")
    print(f"Page: {chunk['page']}")
    print(f"Text (first 200 chars): {chunk['chunk'][:200]}...")
    print(f"Embedding shape: {emb.shape}")
    print(f"Embedding sample (first 10 values): {emb[:10]}")
    print("-" * 70)


🧠 COMPONENT 2: Semantic Layer initializing...
   Loading model: all-MiniLM-L6-v2
   (First run downloads 22MB - takes 30 seconds)
✅ Model loaded! Embeddings dimension: 384

🔄 Encoding 57 document chunks to embeddings...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Embeddings created: (57, 384)

📄 Random preview of 5 chunks with embeddings:

--- Sample 1 (Index 30) ---
Page: 11
Text (first 200 chars): Airflow requirements Power requirements Power requirements Space requirements Space requirements Temperature requirements Temperature requirements Firewall and port requirements Firewall and port requ...
Embedding shape: (384,)
Embedding sample (first 10 values): [ 0.00880686  0.02316867  0.0089285   0.01078988  0.00056803 -0.03762256
 -0.00877929  0.03573978 -0.02206737  0.0099078 ]
----------------------------------------------------------------------
--- Sample 2 (Index 16) ---
Page: 7
Text (first 200 chars): Essentials Software Documentation AI Essentials Software Documentation . The following sections summarize the key features and capabilities of HPE AI Essentials Software. Core Components Core Componen...
Embedding shape: (384,)
Embedding sample (first 10 values): [-0.09123894 -0.00673326 -0.02405057  0.04224906  0.03336014 -0.06206925
  0.

## Component 3: Retrieval System

Build FAISS index for fast similarity search.

In [6]:
# Initialize Retrieval System
retrieval = RetrievalSystem(dimension=384)

# Build searchable index
retrieval.build_index(document_embeddings, kb.chunks)

print("\n✅ Retrieval system ready to search!")

🔍 COMPONENT 3: Retrieval System initialized (dim=384)

🔨 Building FAISS index...
   Documents: 57
   Embedding shape: (57, 384)
✅ Index built with 57 vectors

✅ Retrieval system ready to search!


In [8]:
# Inspect retrieval output
results = retrieval.search(semantic.encode_query("What are pcai core components"), top_k=3)
print("Results structure:\n")
for i, r in enumerate(results):
    print(f"Result {i}:")
    print(r)
    print("-" * 50)



🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents
Results structure:

Result 0:
{'rank': 1, 'document': {'page': 1, 'chunk': 'HPE Private Cloud AI 1.5 Administration Guide HPE Private Cloud AI 1.5 Administration Guide Part Number: 20-PCAI-UG-ED1 Published: September 2025 Edition: 1.5'}, 'similarity': 0.4387816354426174, 'distance': 1.2790379524230957}
--------------------------------------------------
Result 1:
{'rank': 2, 'document': {'page': 3, 'chunk': 'Table of contents Table of contents Overview of the HPE Private Cloud AI engineered system Private Cloud AI Stack Developer System HPE AI Essentials: Core Concepts Solution components Optimum environment Airflow requirements Power requirements Space requirements Temperature requirements Firewall and port requirements Networking requirements Licensing requirements Activating the NVIDIA software subscription Solution configurations Configuration sizes Private Cloud AI User roles Signing in to the HPE GreenLake platfo

## Test the Complete RAG Pipeline

Now let's ask a question and see all 5 components work together!

In [9]:
def ask_question(question, top_k=3):
    """Complete RAG pipeline for a question using PCAI PDF knowledge base"""
    
    print("=" * 70)
    print(f"❓ QUESTION: {question}")
    print("=" * 70)
    
    # COMPONENT 2: Encode query
    print("\n[Step 1] Encoding question...")
    query_embedding = semantic.encode_query(question)
    
    # COMPONENT 3: Retrieve relevant chunks
    print("[Step 2] Searching knowledge base...")
    results = retrieval.search(query_embedding, top_k=top_k)
    
    print(f"\n📚 Retrieved Top {top_k} Chunks:")
    for r in results:
        doc_page = r['document']['page']
        doc_text = r['document']['chunk']
        print(f"  • Page {doc_page} (similarity: {r['similarity']:.1%})")
        print(f"    Text snippet: {doc_text[:200]}...\n")
    
    # COMPONENT 4: Augment with context
    print("[Step 3] Creating enriched context...")
    augmentor = Augmentation()
    enriched = augmentor.create_context(question, results)
    
    # COMPONENT 5: Generate answer
    print("[Step 4] Generating answer...")
    generator = Generation(model_name='llama3.2:3b', use_llm=True)
    answer = generator.generate(enriched, results)
    
    # Display answer
    print("\n" + "=" * 70)
    print("💬 ANSWER:")
    print("=" * 70)
    print(answer)
    print("=" * 70)
    
    return answer

# Example usage
ask_question("What are pcai core tools")



❓ QUESTION: What are pcai core tools

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 7 (similarity: 47.9%)
    Text snippet: HPE HPE AI Essentials AI Essentials : Core Concepts : Core Concepts HPE AI Essentials Software is part of the HPE Private Cloud AI software and data foundation layer that accelerates the time from AI ...

  • Page 8 (similarity: 47.7%)
    Text snippet: AI assistants for the fastest path to inference with unparalleled performance. HPE AI Essentials Software currently includes the following pre-packaged NVIDIA NIM: NVIDIA NIM for GPU accelerated NVIDI...

  • Page 8 (similarity: 47.3%)
    Text snippet: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI applications. Blueprints are designed for you to d...

[Step 3] Creating enriched context...

'Based on the provided context from the PDF, PCAI (not explicitly defined in this text) is not directly mentioned as a core tool. However, there are several AI-related tools and frameworks that are mentioned:\n\n1. NVIDIA NIM (Neural Network Inference Module) - a pre-packaged AI framework for GPU accelerated inference.\n2. NVIDIA Blueprints - predefined, customizable AI workflows from NVIDIA for creating and deploying generative AI applications.\n\nAdditionally, the text mentions "Open-Source Tools & Frameworks" as part of the HPE AI Essentials Software\'s core components, but it does not specifically mention PCAI as one of them.\n\nIf we look at the broader context of the topic, PCAI (Plurality Cognitive Architecture) is a type of generative AI model that was introduced by Plurality in 2022. It is designed to generate human-like text and is often used for tasks such as chatbots, content generation, and language translation.\n\nWhile PCAI is not explicitly mentioned in the provided con

## Try More Questions!

Test the system with different queries.

In [10]:
# Try these questions or write your own!

ask_question("What is pcai engineered system for?")

❓ QUESTION: What is pcai engineered system for?

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 8 (similarity: 46.9%)
    Text snippet: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI applications. Blueprints are designed for you to d...

  • Page 1 (similarity: 45.6%)
    Text snippet: HPE Private Cloud AI 1.5 Administration Guide HPE Private Cloud AI 1.5 Administration Guide Part Number: 20-PCAI-UG-ED1 Published: September 2025 Edition: 1.5...

  • Page 3 (similarity: 45.4%)
    Text snippet: Table of contents Table of contents Overview of the HPE Private Cloud AI engineered system Private Cloud AI Stack Developer System HPE AI Essentials: Core Concepts Solution components Optimum environm...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation in

'Based on the provided context, PCAI (Private Cloud AI) is an engineered system designed for use with NVIDIA Blueprints, specifically for creating and deploying generative AI applications, including building AI virtual assistants and digital twins.\n\n======================================================================\n📚 SOURCES:\n======================================================================\n\n[1] Page 8 (Similarity: 46.9%)\n    Preview: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI ...\n\n[2] Page 1 (Similarity: 45.6%)\n    Preview: HPE Private Cloud AI 1.5 Administration Guide HPE Private Cloud AI 1.5 Administration Guide Part Number: 20-PCAI-UG-ED1 Published: September 2025 Edit...\n\n[3] Page 3 (Similarity: 45.4%)\n    Preview: Table of contents Table of contents Overview of the HPE Private Cloud AI engineered system Private Cloud AI Stack Developer System HPE AI Ess

In [11]:
ask_question("is spark used with livy")

❓ QUESTION: is spark used with livy

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 20 (similarity: 41.1%)
    Text snippet: Virtualization Virtualization VM Essentials Data Service Connectors Data Service Connectors HPE -DSC VM with equivalent KickStart Scripts Data Engineering Tools Data Engineering Tools Apache Airflow A...

  • Page 7 (similarity: 41.1%)
    Text snippet: Essentials Software Documentation AI Essentials Software Documentation . The following sections summarize the key features and capabilities of HPE AI Essentials Software. Core Components Core Componen...

  • Page 19 (similarity: 38.8%)
    Text snippet: support) NIM Mistral 4B Mistral 4B for reranking LORA adapters for model fine-tuning LORA adapters for model fine-tuning Community AI Models and Repositories Aleph Alpha Aleph Alpha , Hugging Face Hug...

[Step 3] Creating enriched context..

'Based on the provided context, it appears that Apache Spark is used with Apache Airflow and EzPresto, but not directly with Livy.\n\nOn Page 7, under "Data Engineering and Analytics Tools", HPE AI Essentials Software mentions:\n\n* Apache Spark: Enterprise-grade analytics engine supporting large-scale data processing and transformation operations.\n* Apache Airflow: Advanced workflow orchestration platform that integrates directly with HPE AI Essentials data sets.\n\nThere is no mention of Livy in this section. However, it\'s worth noting that Apache Airflow can be used to manage and schedule Spark jobs, including those submitted through Livy.\n\nOn Page 20, under "Data Engineering Tools", HPE AI Essentials Software mentions:\n\n* EzPresto: SQL-based query engine optimized for large-scale data analysis.\n* Superset: Enterprise-class business intelligence platform offering advanced data visualization capabilities.\n\nAgain, there is no mention of Livy in this section.\n\nIt\'s possible

In [12]:
# Your own question here!
ask_question("What is the hardware used")

❓ QUESTION: What is the hardware used

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 8 (similarity: 39.1%)
    Text snippet: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI applications. Blueprints are designed for you to d...

  • Page 10 (similarity: 38.4%)
    Text snippet: Entity Entity Component Details Component Details Small, Medium and Large Small, Medium and Large Developer System Developer System Software Built on the HPE GreenLake Cloud Platform GreenLake Cloud P...

  • Page 8 (similarity: 38.4%)
    Text snippet: /mnt/datasources for immediate data access. Implements a shared directory system for team collaboration while maintaining security boundaries. Includes comprehensive tutorials and predictive analysis ...

[Step 3] Creating enriched context.

'The hardware used in the NVIDIA Blueprints and HPE AI Essentials environment is:\n\n1. NVIDIA GPUs (Graphics Processing Units)\n2. HPE GreenLake for File Storage with a minimum capacity of at least 109 TB\n3. Local NFS with 32TB of internal file and object storage.\n\nNote that the specific hardware specifications are not provided in the context, but these are the components mentioned as being used in the environment.\n\n======================================================================\n📚 SOURCES:\n======================================================================\n\n[1] Page 8 (Similarity: 39.1%)\n    Preview: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI ...\n\n[2] Page 10 (Similarity: 38.4%)\n    Preview: Entity Entity Component Details Component Details Small, Medium and Large Small, Medium and Large Developer System Developer System Software Built on ...\n\n[3] Page 8 

## Interactive Mode

Keep asking questions!

In [ ]:
# Interactive questioning
while True:
    user_question = input("\nYour question (or 'quit' to stop): ").strip()
    
    if user_question.lower() in ['quit', 'exit', 'q']:
        print("👋 PDF RAG here to answer")
        break
    
    if user_question:
        ask_question(user_question)
    else:
        print("Please enter a question!")


Your question (or 'quit' to stop):  what are solution components,  how are they used


❓ QUESTION: what are solution components,  how are they used

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 9 (similarity: 43.5%)
    Text snippet: and permission management Features sophisticated data source connection management with security controls Allows administrators to configure and customize included applications through the user interf...

  • Page 10 (similarity: 42.5%)
    Text snippet: Entity Entity Component Details Component Details Small, Medium and Large Small, Medium and Large Developer System Developer System Software Built on the HPE GreenLake Cloud Platform GreenLake Cloud P...

  • Page 8 (similarity: 40.6%)
    Text snippet: NVIDIA Blueprints NVIDIA Blueprints are predefined, customizable AI workflows from NVIDIA that can assist you in creating and deploying generative AI applications. Blueprints are designed for you to d...

[Step 3] Cre